In [1]:
import numpy as np
import pandas as pd
!pip install bs4==0.0.1
import urllib
import bs4
from urllib import request
from tqdm import tqdm


  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=dcef2f6561a80747f36dde00e2ee98764625d68a7f599d9a7b31487197bd0ea1
  Stored in directory: /root/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [5]:

attente = pd.read_csv("data/donnee temps attente (avec numero INSEE).csv")
densite = pd.read_csv("data/donnee densite.csv")
nuitees = pd.read_csv("data/data_nuités_communes.csv")
ville = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/554590ab-ae62-40ac-8353-ee75162c05ee", sep=";")

In [6]:
densite['Numéro'] = densite['Numéro'].apply(lambda x : '0'+x if len(x)==1 else x)

In [7]:
nuitees.set_index("Numéro", inplace = True)
densite.set_index("Numéro", inplace = True)
attente.set_index("Numéro INSEE", inplace = True)
ville.set_index("Code_commune_INSEE", inplace = True)

In [8]:
ville.loc['69123'] = ['LYON', 69001, float('nan'), 'LYON', '45.7577778,4.832222222222222']
ville.loc['13055'] = ['MARSEILLE', 13001, float('nan'), 'MARSEILLE', '43.2963889,5.369999999999999']
ville.loc['75056'] = ['PARIS', 75001, float('nan'), 'PARIS', '48.8566667,2.3519444444444444']

In [9]:
dico_nuitees=dict()
dico_densite=dict()
dico_classement=dict()
erreurs=[]
for idx in tqdm(attente.index):
    dico_nuitees[idx] = list(nuitees.loc[idx[:2]])
    dico_densite[idx] = list(densite.loc[idx[:2]])
    
    try : #certaines grandes villes ont plusieurs codes postaux 
        code_postal=str(ville.loc[idx, 'Code_postal'][0]) #on récupère le code postal de la commune
        nom_ville=str(ville.loc[idx, 'Nom_commune'][0]).lower().replace(" ","")
    except: #d'autres non
        code_postal=str(ville.loc[idx, 'Code_postal']) #on récupère le code postal de la commune
        nom_ville=str(ville.loc[idx, 'Nom_commune']).lower().replace(" ","")
    url_classement="https://www.villesetvillagesouilfaitbonvivre.com/le-palmares/classement/"+idx+"/"+code_postal+"/"+nom_ville+"#city-ranking"
    request_text_2 = request.urlopen(url_classement).read()
    page_2 = bs4.BeautifulSoup(request_text_2, "lxml")
    div_du_classement= list((page_2.find("p",{"class":'h1'})).stripped_strings)
    dico_classement[idx] = div_du_classement[0]
    

100%|██████████| 314/314 [01:00<00:00,  5.16it/s]


In [11]:
import json
fichier_meteo = open("data/prefectures-sousprefectures.json","r")
meteo_dico = json.load(fichier_meteo)
fichier_meteo.close()
meteo = pd.DataFrame(meteo_dico)
meteo.set_index("code_insee", inplace = True)

In [12]:
meteo.loc['44003']=[781.6,float('nan'), 12.7, float('nan'), float('nan')]
meteo.loc['51507']=[857,1682, float('nan'), float('nan'), 15507]

In [14]:
nui = pd.DataFrame(dico_nuitees, index = nuitees.columns).T
del nui['Départements']
den = pd.DataFrame(dico_densite, index = densite.columns).T
del den['Unnamed: 0']
cla = pd.DataFrame(dico_classement, index = ['Classement']).T

In [15]:
total = pd.concat([attente, nui, den, cla, meteo], axis=1)

In [16]:
fichier = open("data/donnees concatenees.csv",'w')
fichier.write(total.to_csv())
fichier.close()

# total.to_csv("data/donnees concatenees.csv")

In [ ]:
import pandas as pd
temps_attente = pd.read_csv("data/donnees concatenees.csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2f812232-8ed1-4e89-8f48-597e10058637' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>